In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch
from torch.utils.data import Dataset, DataLoader

model_name = 'gpt2'
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

In [ ]:
#load the corpus
with open('definitions.txt', 'r') as f:
    corpus = f.read()
    
#tokenize the corpus
tokenized_corpus = tokenizer.encode(corpus)

In [ ]:
#split the tokenized corpus into chunks of tokens
max_length = 128
stride = 64

inputs = []
labels = []
for i in range(0, len(tokenized_corpus) - max_length, stride):
    inputs.append(tokenized_corpus[i:i+max_length])
    labels.append(tokenized_corpus[i+1:i+1+max_length])

#convert the inputs and labels into tensors
inputs = torch.tensor(inputs)
labels = torch.tensor(labels)

In [1]:
#fine-tune the model on custom corpus
class CustomDataset(Dataset):
    def __init__(self, inputs, labels):
        self.inputs = inputs
        self.labels = labels

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        return self.inputs[idx], self.labels[idx]

batch_size = 16
num_epochs = 10
learning_rate = 2e-5

dataset = CustomDataset(inputs, labels)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
loss_fn = torch.nn.CrossEntropyLoss()

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

for epoch in range(num_epochs):
    running_loss = 0.0
    model.train()
    for batch in dataloader:
        optimizer.zero_grad()
        input_ids, labels = batch
        input_ids = input_ids.to(device)
        labels = labels.to(device)
        outputs = model(input_ids, labels=labels)
        loss = loss_fn(outputs.logits.view(-1, outputs.logits.shape[-1]), labels.view(-1))
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print('Epoch {} loss: {:.4f}'.format(epoch+1, running_loss/len(dataloader)))

NameError: name 'Dataset' is not defined

In [ ]:
model.save_pretrained('fine-tuned-gpt2')
tokenizer.save_pretrained('fine-tuned-gpt2')